# Automatic Connection Requests for LinkedIn

This notebook can be used to automatically send requests to people on LinkedIn for specific profiles that we are interested in. The profile name can be assigned into the variable `SEARCH_TERM` available below in the `Initializations` section. It uses the library `selenium` to login to LinkedIn and send connection or follow requests. <br>
*Note: Create a file called `username.py` in some other directory. Next, add that directory to the path variable using the `sys.path.append` and import it*

### Imports

In [ ]:
import logging
import time
import traceback
import os
import sys
import pandas as pd
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
sys.path.append('../../../')
from username import linkedin_username, linkedin_password

### Initializations


In [ ]:
SEARCH_TERM = "Senior Data Scientist"
NUM_PAGES_TO_FETCH_FOR_PROFILES = 3

ERROR_LOG_STRING_FORMAT = '''
    {section_title}
    ---
    {description}
    ---
    {error_message}
''' + '='*60 + '\n\n'

email = linkedin_username
password = linkedin_password

### Helper Functions

In [ ]:
def error_logging_helper(log_filename:str = 'all_logs.log', logger_name:str = 'error_logger'):
    '''
    helper function for logging errors
    '''
    logger = logging.getLogger(logger_name)
    handler = logging.FileHandler(
        filename=log_filename,
        mode='a+'
    )
    log_format = logging.Formatter('%(asctime)s - %(message)s')
    handler.setLevel(logging.INFO)
    handler.setFormatter(log_format)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    print(logger.getEffectiveLevel())

    return logger

logger = error_logging_helper('extras/all_logs.log')

### Enter Username and Password and Click on Login

In [ ]:
### Open Chrome Driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

### Go to linkedin URL
driver.get("https://linkedin.com")

### Enter Username and Password and Click on Login
input_elements = driver.find_elements(
    By.XPATH,
    "//input[contains(@class, 'input__input' )][1]"
)
input_elements[0].send_keys(email)
input_elements[1].send_keys(password)

submit_button = driver.find_elements(
    By.XPATH,
    "//button[contains(@class, 'sign-in-form__submit-button')]"
)
submit_button[0].click()

time.sleep(7)

### Focus on Search Button, Enter Search Term, and Press Enter

In [ ]:
try:
    ### find search button
    search_button = driver.find_elements(
        By.XPATH,
        "//button[contains(@class, 'search-global-typeahead__collapsed-search-button')]"
    )
    search_button[0].click()
    
except:
    ### input search term in the search field
    input_search_field_xpath = "//input[contains(@class, 'search-global-typeahead__input always-show-placeholder')]"
    search_input_field = driver.find_elements(
        By.XPATH, 
        input_search_field_xpath
    )

    search_input_field[0].clear()
    search_input_field[0].send_keys(SEARCH_TERM + "\n")
    
time.sleep(5)

### Click on "See all people results"

In [ ]:
see_all_results = driver.find_elements(
    By.XPATH,
    "//a[contains(text(), 'See all people results')]"
)
see_all_results[0].click()
time.sleep(5)

### Send Connection Requests in a Loop

In [ ]:
for page_num in range(1, NUM_PAGES_TO_FETCH_FOR_PROFILES+1):    
    try:
        ################################
        ### Send Connection Requests ###
        
        try:
            # get all the connect buttons from the page
            connect_buttons = driver.find_elements(
                By.XPATH,
                '''
                //button[starts-with(@aria-label, 'Invite') and 
                (
                substring(
                    @aria-label, string-length(@aria-label) - string-length('to connect')+1
                ) = 'to connect'
                )]
                '''
            )

            if len(connect_buttons) > 0:
                for connect_single_button in connect_buttons:
                    # click on each connect button to open the popup for sending connection request
                    connect_single_button.click()
                    time.sleep(2)

                    send_button = driver.find_elements(
                        By.XPATH,
                        "//button[@aria-label = 'Send now']"
                    )

                    # send connection request when possible
                    # skip scenarios where we cannot directly send connection requests
                    if len(send_button) > 0:
                        send_button[0].click()
                    else:
                        cancel_buttons = driver.find_elements(
                            By.XPATH,
                            "//button[@aria-label = 'Dismiss']"
                        )
                        cancel_buttons[0].click()
                    time.sleep(2)
            
        except Exception as e:
            logger.error(
                ERROR_LOG_STRING_FORMAT.format(
                    section_title = 'Error in Sending Connection Request to Users',
                    description = 'Error for  Page Num: ' + str(page_num),
                    error_message = str(e) + "\n" + str(traceback.format_exc())
                )
            )
            
          
        ################################
        ######### Follow Users #########
        try:
            follow_buttons = driver.find_elements(
                By.XPATH,
                "//button[@aria-label = 'Follow']"
            )

            # follow users that we cannot directly send a connection request to but have an option to follow
            if len(follow_buttons) > 0:
                for follow_single_button in follow_buttons:
                    follow_single_button.click()
    
        except Exception as e:
            logger.error(
                ERROR_LOG_STRING_FORMAT.format(
                    section_title = 'Error in Following Users',
                    description = 'Error for  Page Num: ' + str(page_num),
                    error_message = str(e) + "\n" + str(traceback.format_exc())
                )
            )
                
                
        ###################################
        ######### Click Next Page #########   
        time.sleep(3)
        # scroll the are where the next button will be available
        driver.refresh()
        driver.execute_script("window.scrollTo(0, 0);")
        driver.execute_script("window.scrollTo(0, 1100);")
        time.sleep(2)
        # click on next and wait for sometime
        next_button = driver.find_elements(
            By.XPATH,
            "//button[contains(@class, 'artdeco-pagination__button--next')]"
        )[0]
        next_button.click()
        time.sleep(5)

    except Exception as e:
        logger.error(
            ERROR_LOG_STRING_FORMAT.format(
                section_title = 'Error in Sending Requests or Clicking Next Page',
                description = 'Error for  Page Num: ' + str(page_num),
                error_message = str(e) + "\n" + str(traceback.format_exc())
            )
        )

In [ ]:
driver.close()